# Seasonal Decomposition (Classical)

## Purpose
- Separate a series into trend, seasonal, and residual components.
- Build intuition for how classical models explain structure.
- Create clean baselines before more complex models.

## When to use it
- Clear, repeating seasonality with a known period (daily, weekly, monthly).
- A relatively smooth trend plus noise.
- You want an interpretable breakdown of what drives the series.


## Additive vs multiplicative models

Additive (constant seasonal amplitude):
$$y_t = T_t + S_t + R_t$$

Multiplicative (seasonality scales with the level):
$$y_t = T_t \times S_t \times R_t$$

A common trick for multiplicative series is to take logs:
$$\log y_t = \log T_t + \log S_t + \log R_t$$


## Classical decomposition workflow
1. Choose a seasonal period (e.g., 12 for monthly data with yearly seasonality).
2. Estimate the trend using a moving average.
3. Remove the trend and average by season to estimate the seasonal pattern.
4. Compute residuals as what remains after removing trend and seasonality.

This section builds a simple additive decomposition from scratch.


In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

rng = np.random.default_rng(42)

period = 12
n = 144
t = np.arange(n)
trend = 0.03 * t
seasonal = 1.5 * np.sin(2 * np.pi * t / period) + 0.4 * np.cos(2 * np.pi * t / period)
noise = rng.normal(0, 0.4, n)
y = trend + seasonal + noise

index = pd.date_range("2012-01-01", periods=n, freq="M")
series = pd.Series(y, index=index, name="y")

fig = go.Figure()
fig.add_trace(go.Scatter(x=series.index, y=series, name="observed"))
fig.update_layout(title="Synthetic series with trend + seasonality", xaxis_title="date", yaxis_title="value")
fig

In [ ]:
def classical_decompose(series: pd.Series, period: int, model: str = "additive") -> pd.DataFrame:
    """Simple classical decomposition with moving-average trend.

    Returns a DataFrame with columns: trend, seasonal, resid.
    """
    if model not in {"additive", "multiplicative"}:
        raise ValueError("model must be 'additive' or 'multiplicative'")

    y = series.astype(float)
    trend = y.rolling(window=period, center=True, min_periods=period // 2).mean()

    if model == "multiplicative":
        detrended = y / trend
    else:
        detrended = y - trend

    season_index = np.arange(len(y)) % period
    seasonal_means = detrended.groupby(season_index).mean()
    seasonal = pd.Series(seasonal_means.iloc[season_index].to_numpy(), index=y.index)

    if model == "multiplicative":
        seasonal = seasonal / np.nanmean(seasonal)
        resid = y / (trend * seasonal)
    else:
        seasonal = seasonal - np.nanmean(seasonal)
        resid = y - trend - seasonal

    return pd.DataFrame({"trend": trend, "seasonal": seasonal, "resid": resid})

components = classical_decompose(series, period=period, model="additive")
components.head()


In [ ]:
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.03)

fig.add_trace(go.Scatter(x=series.index, y=series, name="observed"), row=1, col=1)
fig.add_trace(go.Scatter(x=series.index, y=components["trend"], name="trend"), row=2, col=1)
fig.add_trace(go.Scatter(x=series.index, y=components["seasonal"], name="seasonal"), row=3, col=1)
fig.add_trace(go.Scatter(x=series.index, y=components["resid"], name="residual"), row=4, col=1)

fig.update_layout(height=900, title="Classical additive decomposition")
fig.update_yaxes(title_text="value", row=1, col=1)
fig.update_yaxes(title_text="trend", row=2, col=1)
fig.update_yaxes(title_text="seasonal", row=3, col=1)
fig.update_yaxes(title_text="residual", row=4, col=1)
fig

## Interpreting the components
- Trend captures long-run movement (smoothed with a moving average).
- Seasonal repeats every period; it should be stable over time.
- Residuals are what the model cannot explain. If residuals still show patterns, the model is missing structure.

Edge effects: the moving average uses a window, so the trend is less reliable at the beginning and end.


In [ ]:
resid = components["resid"].dropna()
fig = go.Figure()
fig.add_trace(go.Histogram(x=resid, nbinsx=40, name="residuals"))
fig.update_layout(title="Residual distribution (should be centered near 0)", xaxis_title="residual")
fig.show()

print("Residual mean:", resid.mean())
print("Residual std:", resid.std())


## Exercises
- Change the seasonal period and see how the decomposition reacts.
- Try multiplicative mode on a strictly positive series.
- Add a second seasonal pattern and observe how the residuals change.


## Further reading
- Hyndman and Athanasopoulos, Forecasting: Principles and Practice (decomposition chapter).
- statsmodels seasonal_decompose and STL documentation.
